In [1]:
import os
print(os.getcwd())

h:\Work_With_MZ\LC_Classification_Code


In [2]:
import os

# Change to your project directory (update the path as needed)
os.chdir(r"H:\Work_With_MZ\LC_Classification_Code")

# Verify the change
print(os.getcwd())

h:\Work_With_MZ\LC_Classification_Code


In [3]:
# Import libraries
import ee
import geemap
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import folium
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [5]:
# Trigger the authentication flow
ee.Authenticate()

True

In [6]:
# Initialize the library
ee.Initialize()

In [7]:
# Initializes the Google Earth Engine API
geemap.ee_initialize()

In [1]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import folium
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from shapely.geometry import Point, Polygon
from shapely.validation import make_valid
import json
from shapely.geometry import mapping
from shapely import geometry

# 1. Authentication and Initialization
def initialize_earth_engine():
    """Initialize Earth Engine with robust error handling."""
    try:
        ee.Initialize()
        print("Earth Engine initialized successfully.")
        return True
    except Exception as e:
        print(f"Initial attempt to initialize Earth Engine failed: {e}")
        print("Attempting to authenticate...")
        try:
            ee.Authenticate()
            ee.Initialize()
            print("Earth Engine authenticated and initialized successfully.")
            return True
        except Exception as auth_e:
            print(f"Authentication failed: {auth_e}")
            print("Please ensure you have an active Google Earth Engine account and valid credentials.")
            print("Run `earthengine authenticate` in a terminal or follow the browser prompts.")
            return False

if not initialize_earth_engine():
    print("Earth Engine initialization failed. Exiting.")
    exit()

# 2. Load the study area boundary and water observation data
study_area_path = "Province_Sofian/Export_Output.shp"
water_areas_path = "Pools_Observation/Pools.shp"

try:
    study_area_gdf = gpd.read_file(study_area_path)
    water_areas_gdf = gpd.read_file(water_areas_path)
except Exception as e:
    print(f"Error loading shapefiles: {e}")
    print("Please ensure the file paths are correct and shapefiles are valid.")
    exit()

study_area_gdf = study_area_gdf.to_crs(epsg=4326)
water_areas_gdf = water_areas_gdf.to_crs(epsg=4326)

def remove_z_coordinate(geom):
    """Convert 3D geometry to 2D by removing z-coordinate."""
    if geom is None:
        return None
    if geom.has_z:
        if geom.geom_type == 'Polygon':
            coords = [list(coord)[0:2] for coord in geom.exterior.coords]
            return Polygon(coords)
        elif geom.geom_type == 'MultiPolygon':
            polygons = []
            for poly in geom.geoms:
                coords = [list(coord)[0:2] for coord in poly.exterior.coords]
                polygons.append(Polygon(coords))
            return geometry.MultiPolygon(polygons)
    return geom

print("Converting 3D geometries to 2D...")
study_area_gdf['geometry'] = study_area_gdf.geometry.apply(remove_z_coordinate)
water_areas_gdf['geometry'] = water_areas_gdf.geometry.apply(remove_z_coordinate)

print("Validating and cleaning geometries...")
invalid_geometries = water_areas_gdf[~water_areas_gdf.geometry.is_valid]
empty_geometries = water_areas_gdf[water_areas_gdf.geometry.is_empty]
print(f"Invalid geometries in water_areas_gdf: {len(invalid_geometries)}")
print(f"Empty geometries in water_areas_gdf: {len(empty_geometries)}")

water_areas_gdf = water_areas_gdf[water_areas_gdf.geometry.is_valid & ~water_areas_gdf.geometry.is_empty]
print(f"Number of features in water_areas_gdf after cleaning: {len(water_areas_gdf)}")
print(f"Geometry types in water_areas_gdf: {water_areas_gdf.geometry.type.unique()}")

water_areas_gdf['geometry'] = water_areas_gdf.geometry.simplify(tolerance=0.0001, preserve_topology=True)
water_areas_gdf = water_areas_gdf.explode(index_parts=True).reset_index(drop=True)
water_areas_gdf['geometry'] = water_areas_gdf.geometry.apply(make_valid)

invalid_geometries_study = study_area_gdf[~study_area_gdf.geometry.is_valid]
empty_geometries_study = study_area_gdf[study_area_gdf.geometry.is_empty]
print(f"Invalid geometries in study_area_gdf: {len(invalid_geometries_study)}")
print(f"Empty geometries in study_area_gdf: {len(empty_geometries_study)}")
study_area_gdf = study_area_gdf[study_area_gdf.geometry.is_valid & ~study_area_gdf.geometry.is_empty]
study_area_gdf['geometry'] = study_area_gdf.geometry.simplify(tolerance=0.0001, preserve_topology=True)
study_area_gdf['geometry'] = study_area_gdf.geometry.apply(make_valid)
print(f"Number of features in study_area_gdf after cleaning: {len(study_area_gdf)}")

center_point = study_area_gdf.geometry.unary_union.centroid
map_center = [center_point.y, center_point.x]
print(f"The center point of your study area is: Latitude={map_center[0]:.4f}, Longitude={map_center[1]:.4f}")

try:
    roi = geemap.gdf_to_ee(study_area_gdf)
    print("Successfully converted study_area_gdf to Earth Engine FeatureCollection.")
except Exception as e:
    print(f"Error converting study_area_gdf to EE FeatureCollection: {e}")
    temp_geojson = "temp_study_area.geojson"
    try:
        study_area_gdf.to_file(temp_geojson, driver="GeoJSON")
        with open(temp_geojson, 'r') as f:
            geojson_data = json.load(f)
        for feature in geojson_data['features']:
            if feature['geometry']['type'] == 'Polygon':
                feature['geometry']['coordinates'] = [[list(coord)[0:2] for coord in ring] for ring in feature['geometry']['coordinates']]
            elif feature['geometry']['type'] == 'MultiPolygon':
                feature['geometry']['coordinates'] = [[[list(coord)[0:2] for coord in ring] for ring in poly] for poly in feature['geometry']['coordinates']]
        roi = ee.FeatureCollection(geojson_data)
        print("Manually converted study_area_gdf GeoJSON to Earth Engine FeatureCollection.")
    except Exception as e:
        print(f"Manual conversion of study_area_gdf failed: {e}")
        print("Sample study_area_gdf feature:")
        if geojson_data.get('features'):
            print(json.dumps(geojson_data['features'][0], indent=2))
        exit()

water_polygons_ee = None
try:
    water_polygons_ee = geemap.gdf_to_ee(water_areas_gdf)
    print("Successfully converted water_areas_gdf to Earth Engine FeatureCollection.")
except Exception as e:
    print(f"Error converting water_areas_gdf to EE FeatureCollection: {e}")
    print("Attempting manual GeoJSON conversion...")
    temp_geojson = "temp_water_areas.geojson"
    try:
        water_areas_gdf.to_file(temp_geojson, driver="GeoJSON")
        with open(temp_geojson, 'r') as f:
            geojson_data = json.load(f)
        for feature in geojson_data['features']:
            if feature['geometry']['type'] == 'Polygon':
                feature['geometry']['coordinates'] = [[list(coord)[0:2] for coord in ring] for ring in feature['geometry']['coordinates']]
            elif feature['geometry']['type'] == 'MultiPolygon':
                feature['geometry']['coordinates'] = [[[list(coord)[0:2] for coord in ring] for ring in poly] for poly in feature['geometry']['coordinates']]
        water_polygons_ee = ee.FeatureCollection(geojson_data)
        print("Manually converted water_areas_gdf GeoJSON to Earth Engine FeatureCollection.")
    except Exception as e:
        print(f"Manual GeoJSON conversion failed: {e}")
        print("Sample water_areas_gdf feature:")
        if geojson_data.get('features'):
            print(json.dumps(geojson_data['features'][0], indent=2))
        exit()

if water_polygons_ee is None:
    print("Error: water_polygons_ee could not be created. Exiting.")
    exit()

# 3. Data Downloading and Processing
start_date = "2023-01-01"
end_date = "2023-12-31"

def s2Process(dates, extent, cloud):
    try:
        start_date, end_date = dates
        def imgDownload():
            img_collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                        .filterDate(start_date, end_date)
                        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", cloud))
                        .filterBounds(extent))
            size = img_collection.size().getInfo()
            print(f"There are {size} Sentinel-2 Images")
            if size == 0:
                raise Exception("No Sentinel-2 images found for the specified date range and region.")
            return img_collection
        
        def s2ClearSky(image):
            scl = image.select('SCL')
            clear_sky_pixels = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(11))
            return image.updateMask(clear_sky_pixels).multiply(0.0001)
        
        def bandResample(image):
            bands_10m = image.select(["B2", "B3", "B4", "B8"])
            bands_20m = image.select(["B11"]).resample("bilinear").reproject(crs=bands_10m.projection(), scale=10)
            return bands_10m.addBands(bands_20m)

        def indices(image):
            ndvi = image.normalizedDifference(["B8", "B4"]).rename("ndvi")
            mndwi = image.normalizedDifference(["B3", "B11"]).rename("mndwi")
            return image.addBands([ndvi, mndwi])
        
        s2_collection = imgDownload()
        s2_collection_resampled = (s2_collection
                               .map(s2ClearSky)
                               .map(bandResample)
                               .map(indices))
        s2_median = s2_collection_resampled.median().clip(extent)
        
        print(f"The bands in s2_median are: {s2_median.bandNames().getInfo()}")
        return s2_median
    except Exception as e:
        print(f"Error in Sentinel-2 processing: {e}")
        raise

def s1Process(dates, extent, radius):
    try:
        start_date, end_date = dates
        def imgdownload():
            img_collection = (ee.ImageCollection("COPERNICUS/S1_GRD")
                    .filterDate(start_date, end_date)
                    .filter(ee.Filter.eq("instrumentMode", "IW"))
                    .filterMetadata("resolution_meters", "equals", 10)
                    .filterBounds(extent)
                    .select(["VV", "VH"]))
            size = img_collection.size().getInfo()
            print(f"There are {size} Sentinel-1 Images")
            if size == 0:
                raise Exception("No Sentinel-1 images found for the specified date range and region.")
            return img_collection

        def speckle(image):
            image_filtered = image.focal_median(radius, "circle", "meters")
            return image_filtered.copyProperties(image, image.propertyNames())
        
        def ratio(image):
            vh = image.select("VH")
            vv = image.select("VV")
            ratio = vh.divide(vv).rename("VH_VV_ratio")
            return image.addBands([ratio])

        s1_collection = imgdownload()
        s1_collection_filtered = (s1_collection
                                  .map(speckle)
                                  .map(ratio))
        s1_median = s1_collection_filtered.median().clip(extent)
        
        print(f"The bands in s1_median are: {s1_median.bandNames().getInfo()}")
        return s1_median
    except Exception as e:
        print(f"Error in Sentinel-1 processing: {e}")
        raise

try:
    s2_median_2023 = s2Process(dates=(start_date, end_date), extent=roi, cloud=50)  # Increased cloud cover threshold
    s1_median_2023 = s1Process(dates=(start_date, end_date), extent=roi, radius=50)
    stacked_layer = s2_median_2023.select(['ndvi', 'mndwi']).addBands(s1_median_2023.select(['VH', 'VV', 'VH_VV_ratio']))
    print(f"The bands in the stacked layer are: {stacked_layer.bandNames().getInfo()}")
except Exception as e:
    print(f"Error processing Sentinel images: {e}")
    exit()

# 4. Generate Training and Testing Samples
def generate_non_water_samples(roi, water_polygons, num_points):
    """Generate random non-water points within ROI but outside water polygons."""
    try:
        points = ee.FeatureCollection([])
        while points.size().getInfo() < num_points:
            random_points = ee.FeatureCollection.randomPoints(
                region=roi.geometry(), 
                points=num_points, 
                seed=np.random.randint(1, 100000)
            )
            non_water_points = random_points.filter(ee.Filter.geometry(water_polygons.geometry().dissolve()).Not())
            non_water_points_classified = non_water_points.map(lambda f: f.set('Class', 0))
            points = points.merge(non_water_points_classified)
        return points.limit(num_points)
    except Exception as e:
        print(f"Error generating non-water samples: {e}")
        raise

num_samples = 1000  # Increased to 1000 per class

try:
    water_samples = water_polygons_ee.map(lambda f: f.set('Class', 1)).randomColumn('random').limit(num_samples)
    print(f"Water samples created (labeled as 1): {water_samples.size().getInfo()}")
except Exception as e:
    print(f"Error assigning class to water samples: {e}")
    exit()

try:
    non_water_samples = generate_non_water_samples(roi, water_polygons_ee, num_samples)
    print(f"Non-water samples created (labeled as 0): {non_water_samples.size().getInfo()}")
except Exception as e:
    print(f"Error generating non-water samples: {e}")
    exit()

try:
    combined_samples = water_samples.merge(non_water_samples)
    print(f"Total training samples created: {combined_samples.size().getInfo()}")
except Exception as e:
    print(f"Error combining samples: {e}")
    exit()

def trainTest(sample, image, threshold):
    try:
        sample_random = sample.randomColumn("random")
        training_set_features = sample_random.filter(ee.Filter.lt("random", threshold))
        testing_set_features = sample_random.filter(ee.Filter.gte("random", threshold))
        
        def sample_point(feature):
            class_value = feature.get('Class')
            sampled_values = image.reduceRegion(
                reducer=ee.Reducer.first(),
                geometry=feature.geometry(),
                scale=20  # Increased scale to reduce null values
            )
            return ee.Feature(None, sampled_values).set('Class', class_value)

        training = training_set_features.map(sample_point)
        testing = testing_set_features.map(sample_point)

        print(f"Training set size: {training.size().getInfo()}")
        print(f"Testing set size: {testing.size().getInfo()}")

        return training, testing
    except Exception as e:
        print(f"Error splitting training/testing sets: {e}")
        raise

def filter_invalid_samples(feature_collection, band_names, class_property='Class'):
    all_properties_to_check = band_names + [class_property]
    combined_filter = ee.Filter.And(
        [ee.Filter.notNull([prop]) for prop in all_properties_to_check]
    )
    filtered = feature_collection.filter(combined_filter)
    counts = filtered.aggregate_histogram(class_property).getInfo()
    print(f"Class distribution after filtering: {counts}")
    if len(counts) < 2:
        raise ValueError(f"Filtered dataset contains only one class: {counts}")
    return filtered

try:
    training, testing = trainTest(combined_samples, stacked_layer, 0.7)
    band_names = stacked_layer.bandNames().getInfo()
    training = filter_invalid_samples(training, band_names)
    testing = filter_invalid_samples(testing, band_names)
    print("Training set class distribution:", training.aggregate_histogram('Class').getInfo())
    print("Testing set class distribution:", testing.aggregate_histogram('Class').getInfo())
except Exception as e:
    print(f"Error in trainTest: {e}")
    exit()

# 5. Classification and Accuracy Assessment
def rf_svm_classify(classifier_type, parameters, image, training, testing, out_folder, out_file_base):
    try:
        if not os.path.exists(out_folder):
            os.makedirs(out_folder)
        
        error_file_name = os.path.join(out_folder, out_file_base + "error_matrix" + ".csv")
        
        if classifier_type == "rf":
            trees, split, leaf, bag, node, seed = parameters
            model = ee.Classifier.smileRandomForest(
                numberOfTrees=trees,
                variablesPerSplit=split,
                minLeafPopulation=leaf,
                bagFraction=bag, 
                maxNodes=node,
                seed=seed
            )
            classifier_name = "Random Forest"
        elif classifier_type == "svm":
            kernelType, gamma, cost = parameters
            model = ee.Classifier.libsvm(
                kernelType=kernelType,
                gamma=gamma,
                cost=cost
            )
            classifier_name = "SVM"
        
        classifier = model.train(
            features=training, 
            classProperty="Class", 
            inputProperties=image.bandNames()
        )

        validation = testing.classify(classifier)
        
        prediction_counts = validation.aggregate_histogram('classification').getInfo()
        print(f"Prediction distribution for {classifier_name}: {prediction_counts}")
        if len(prediction_counts) < 2:
            raise ValueError(f"Classifier predicted only one class: {prediction_counts}")

        error_matrix = validation.errorMatrix(actual="Class", predicted="classification")
        error_matrix_array = error_matrix.array().getInfo()
        print(f"Raw error matrix array for {classifier_name}:", error_matrix_array)
        
        if len(error_matrix_array) < 2 or any(len(row) < 2 for row in error_matrix_array):
            raise ValueError(f"Error matrix has unexpected shape: {error_matrix_array}")
        
        columns = ["Non-Water", "Water"]
        try:
            data = error_matrix_array
            error_matrix_df = pd.DataFrame(data, columns=columns, index=columns)
        except Exception as e:
            print(f"Error creating DataFrame: {e}")
            print("Falling back to raw error matrix data.")
            error_matrix_df = pd.DataFrame(error_matrix_array)
            
        print(f"\nConfusion matrix for {classifier_name}:\n", error_matrix_df)
        print(f"Overall Accuracy for {classifier_name}:", round(error_matrix.accuracy().getInfo(), 3)) 
        print(f"Kappa for {classifier_type}:", round(error_matrix.kappa().getInfo(), 3))
        
        error_matrix_df.to_csv(error_file_name)
        print(f"Error matrix saved to {error_file_name}")
        
        return image.classify(classifier), error_matrix_df
    except Exception as e:
        print(f"Error in {classifier_type} classification: {e}")
        raise

rf_classification = None
rf_parameters = (50, None, 1, 0.5, None, 0)
try:
    rf_classification, rf_confusion_matrix = rf_svm_classify(
        "rf", 
        rf_parameters, 
        stacked_layer, 
        training, 
        testing, 
        "Outputs", 
        "rf_binary_classification_"
    )
except Exception as e:
    print(f"Error in Random Forest classification: {e}")
    exit()

if rf_classification is None:
    print("Error: rf_classification could not be created. Exiting.")
    exit()

svm_classification = None
svm_parameters = ('RBF', 0.1, 1)
try:
    svm_classification, svm_confusion_matrix = rf_svm_classify(
        "svm", 
        svm_parameters, 
        stacked_layer, 
        training, 
        testing, 
        "Outputs", 
        "svm_binary_classification_"
    )
except Exception as e:
    print(f"Error in SVM classification: {e}")
    exit()

if svm_classification is None:
    print("Error: svm_classification could not be created. Exiting.")
    exit()

# 6. Visualize the final classification and original data
palette_ground_truth = ['b4b4b4', '0000ff']  # Gray for Non-Water, Blue for Ground Truth Water
palette_rf = ['b4b4b4', '00ffff']  # Gray for Non-Water, Cyan for RF Water
palette_svm = ['b4b4b4', '00ff00']  # Gray for Non-Water, Green for SVM Water
vis_params_ground_truth = {'min': 0, 'max': 1, 'palette': palette_ground_truth}
vis_params_rf = {'min': 0, 'max': 1, 'palette': palette_rf}
vis_params_svm = {'min': 0, 'max': 1, 'palette': palette_svm}

try:
    Map = geemap.Map(center=map_center, zoom=10)
    
    Map.addLayer(water_polygons_ee, {'color': '0000ff', 'fillColor': '0000ff', 'opacity': 0.5}, "Ground Truth Water")
    Map.addLayer(rf_classification, vis_params_rf, "Water Classification (RF)")
    Map.addLayer(svm_classification, vis_params_svm, "Water Classification (SVM)")
    Map.addLayer(roi.geometry(), {'color': 'ff0000', 'fillColor': '00000000'}, "Study Area Boundary")
    Map.add_basemap("SATELLITE")
    Map.add_legend(
        title="Water Classification",
        legend_dict={
            'Non-Water': 'b4b4b4',
            'Ground Truth Water': '0000ff',
            'RF Predicted Water': '00ffff',
            'SVM Predicted Water': '00ff00'
        }
    )
    Map
except Exception as e:
    print(f"Error visualizing map: {e}")
    exit()

Earth Engine initialized successfully.
Converting 3D geometries to 2D...
Validating and cleaning geometries...
Invalid geometries in water_areas_gdf: 0
Empty geometries in water_areas_gdf: 0
Number of features in water_areas_gdf after cleaning: 1078
Geometry types in water_areas_gdf: ['Polygon']
Invalid geometries in study_area_gdf: 0
Empty geometries in study_area_gdf: 0
Number of features in study_area_gdf after cleaning: 1
The center point of your study area is: Latitude=38.2285, Longitude=45.9499
Successfully converted study_area_gdf to Earth Engine FeatureCollection.
Successfully converted water_areas_gdf to Earth Engine FeatureCollection.
There are 103 Sentinel-2 Images
The bands in s2_median are: ['B2', 'B3', 'B4', 'B8', 'B11', 'ndvi', 'mndwi']
There are 59 Sentinel-1 Images
The bands in s1_median are: ['VV', 'VH', 'VH_VV_ratio']
The bands in the stacked layer are: ['ndvi', 'mndwi', 'VH', 'VV', 'VH_VV_ratio']
Water samples created (labeled as 1): 1000
Non-water samples created (

In [ ]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import folium
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from shapely.geometry import Point, Polygon
from shapely.validation import make_valid
import json
from shapely.geometry import mapping
from shapely import geometry
import time

# 1. Authentication and Initialization
def initialize_earth_engine():
    """Initialize Earth Engine with robust error handling."""
    try:
        ee.Initialize()
        print("Earth Engine initialized successfully.")
        return True
    except Exception as e:
        print(f"Initial attempt to initialize Earth Engine failed: {e}")
        print("Attempting to authenticate...")
        try:
            ee.Authenticate()
            ee.Initialize()
            print("Earth Engine authenticated and initialized successfully.")
            return True
        except Exception as auth_e:
            print(f"Authentication failed: {auth_e}")
            print("Please ensure you have an active Google Earth Engine account and valid credentials.")
            print("Run `earthengine authenticate` in a terminal or follow the browser prompts.")
            return False

if not initialize_earth_engine():
    print("Earth Engine initialization failed. Exiting.")
    exit()

# 2. Load the study area boundary and water observation data
study_area_path = "Province_Sofian/Export_Output.shp"
water_areas_path = "Pools_Observation/Pools.shp"

try:
    study_area_gdf = gpd.read_file(study_area_path)
    water_areas_gdf = gpd.read_file(water_areas_path)
except Exception as e:
    print(f"Error loading shapefiles: {e}")
    print("Please ensure the file paths are correct and shapefiles are valid.")
    exit()

study_area_gdf = study_area_gdf.to_crs(epsg=4326)
water_areas_gdf = water_areas_gdf.to_crs(epsg=4326)

def remove_z_coordinate(geom):
    """Convert 3D geometry to 2D by removing z-coordinate."""
    if geom is None:
        return None
    if geom.has_z:
        if geom.geom_type == 'Polygon':
            coords = [list(coord)[0:2] for coord in geom.exterior.coords]
            return Polygon(coords)
        elif geom.geom_type == 'MultiPolygon':
            polygons = []
            for poly in geom.geoms:
                coords = [list(coord)[0:2] for coord in poly.exterior.coords]
                polygons.append(Polygon(coords))
            return geometry.MultiPolygon(polygons)
    return geom

print("Converting 3D geometries to 2D...")
study_area_gdf['geometry'] = study_area_gdf.geometry.apply(remove_z_coordinate)
water_areas_gdf['geometry'] = water_areas_gdf.geometry.apply(remove_z_coordinate)

print("Validating and cleaning geometries...")
invalid_geometries = water_areas_gdf[~water_areas_gdf.geometry.is_valid]
empty_geometries = water_areas_gdf[water_areas_gdf.geometry.is_empty]
print(f"Invalid geometries in water_areas_gdf: {len(invalid_geometries)}")
print(f"Empty geometries in water_areas_gdf: {len(empty_geometries)}")

water_areas_gdf = water_areas_gdf[water_areas_gdf.geometry.is_valid & ~water_areas_gdf.geometry.is_empty]
print(f"Number of features in water_areas_gdf after cleaning: {len(water_areas_gdf)}")
print(f"Geometry types in water_areas_gdf: {water_areas_gdf.geometry.type.unique()}")

water_areas_gdf['geometry'] = water_areas_gdf.geometry.simplify(tolerance=0.0001, preserve_topology=True)
water_areas_gdf = water_areas_gdf.explode(index_parts=True).reset_index(drop=True)
water_areas_gdf['geometry'] = water_areas_gdf.geometry.apply(make_valid)

invalid_geometries_study = study_area_gdf[~study_area_gdf.geometry.is_valid]
empty_geometries_study = study_area_gdf[study_area_gdf.geometry.is_empty]
print(f"Invalid geometries in study_area_gdf: {len(invalid_geometries_study)}")
print(f"Empty geometries in study_area_gdf: {len(empty_geometries_study)}")
study_area_gdf = study_area_gdf[study_area_gdf.geometry.is_valid & ~study_area_gdf.geometry.is_empty]
study_area_gdf['geometry'] = study_area_gdf.geometry.simplify(tolerance=0.0001, preserve_topology=True)
study_area_gdf['geometry'] = study_area_gdf.geometry.apply(make_valid)
print(f"Number of features in study_area_gdf after cleaning: {len(study_area_gdf)}")

center_point = study_area_gdf.geometry.unary_union.centroid
map_center = [center_point.y, center_point.x]
print(f"The center point of your study area is: Latitude={map_center[0]:.4f}, Longitude={map_center[1]:.4f}")

try:
    roi = geemap.gdf_to_ee(study_area_gdf)
    print("Successfully converted study_area_gdf to Earth Engine FeatureCollection.")
except Exception as e:
    print(f"Error converting study_area_gdf to EE FeatureCollection: {e}")
    temp_geojson = "temp_study_area.geojson"
    try:
        study_area_gdf.to_file(temp_geojson, driver="GeoJSON")
        with open(temp_geojson, 'r') as f:
            geojson_data = json.load(f)
        for feature in geojson_data['features']:
            if feature['geometry']['type'] == 'Polygon':
                feature['geometry']['coordinates'] = [[list(coord)[0:2] for coord in ring] for ring in feature['geometry']['coordinates']]
            elif feature['geometry']['type'] == 'MultiPolygon':
                feature['geometry']['coordinates'] = [[[list(coord)[0:2] for coord in ring] for ring in poly] for poly in feature['geometry']['coordinates']]
        roi = ee.FeatureCollection(geojson_data)
        print("Manually converted study_area_gdf GeoJSON to Earth Engine FeatureCollection.")
    except Exception as e:
        print(f"Manual conversion of study_area_gdf failed: {e}")
        print("Sample study_area_gdf feature:")
        if geojson_data.get('features'):
            print(json.dumps(geojson_data['features'][0], indent=2))
        exit()

water_polygons_ee = None
try:
    water_polygons_ee = geemap.gdf_to_ee(water_areas_gdf)
    print("Successfully converted water_areas_gdf to Earth Engine FeatureCollection.")
except Exception as e:
    print(f"Error converting water_areas_gdf to EE FeatureCollection: {e}")
    print("Attempting manual GeoJSON conversion...")
    temp_geojson = "temp_water_areas.geojson"
    try:
        water_areas_gdf.to_file(temp_geojson, driver="GeoJSON")
        with open(temp_geojson, 'r') as f:
            geojson_data = json.load(f)
        for feature in geojson_data['features']:
            if feature['geometry']['type'] == 'Polygon':
                feature['geometry']['coordinates'] = [[list(coord)[0:2] for coord in ring] for ring in feature['geometry']['coordinates']]
            elif feature['geometry']['type'] == 'MultiPolygon':
                feature['geometry']['coordinates'] = [[[list(coord)[0:2] for coord in ring] for ring in poly] for poly in feature['geometry']['coordinates']]
        water_polygons_ee = ee.FeatureCollection(geojson_data)
        print("Manually converted water_areas_gdf GeoJSON to Earth Engine FeatureCollection.")
    except Exception as e:
        print(f"Manual GeoJSON conversion failed: {e}")
        print("Sample water_areas_gdf feature:")
        if geojson_data.get('features'):
            print(json.dumps(geojson_data['features'][0], indent=2))
        exit()

if water_polygons_ee is None:
    print("Error: water_polygons_ee could not be created. Exiting.")
    exit()

# Calculate the number of 10m pixels observed as water
print("Calculating the number of 10m pixels observed as water...")
water_image = ee.Image(0).paint(water_polygons_ee, 1).clip(roi)
M = water_image.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=roi.geometry(),
    scale=10,
    maxPixels=1e9
).get('constant').getInfo()
M = int(M)
print(f"Number of 10m pixels observed as water: {M}")

# 3. Data Downloading and Processing
start_date = "2023-01-01"
end_date = "2023-12-31"

def s2Process(dates, extent, cloud):
    try:
        start_date, end_date = dates
        def imgDownload():
            img_collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                        .filterDate(start_date, end_date)
                        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", cloud))
                        .filterBounds(extent))
            size = img_collection.size().getInfo()
            print(f"There are {size} Sentinel-2 Images")
            if size == 0:
                raise Exception("No Sentinel-2 images found for the specified date range and region.")
            return img_collection
        
        def s2ClearSky(image):
            scl = image.select('SCL')
            clear_sky_pixels = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(11))
            return image.updateMask(clear_sky_pixels).multiply(0.0001)
        
        def bandResample(image):
            bands_10m = image.select(["B2", "B3", "B4", "B8"])
            bands_20m = image.select(["B11"]).resample("bilinear").reproject(crs=bands_10m.projection(), scale=10)
            return bands_10m.addBands(bands_20m)

        def indices(image):
            ndvi = image.normalizedDifference(["B8", "B4"]).rename("ndvi")
            mndwi = image.normalizedDifference(["B3", "B11"]).rename("mndwi")
            return image.addBands([ndvi, mndwi])
        
        s2_collection = imgDownload()
        s2_collection_resampled = (s2_collection
                               .map(s2ClearSky)
                               .map(bandResample)
                               .map(indices))
        s2_median = s2_collection_resampled.median().clip(extent)
        
        print(f"The bands in s2_median are: {s2_median.bandNames().getInfo()}")
        return s2_median
    except Exception as e:
        print(f"Error in Sentinel-2 processing: {e}")
        raise

def s1Process(dates, extent, radius):
    try:
        start_date, end_date = dates
        def imgdownload():
            img_collection = (ee.ImageCollection("COPERNICUS/S1_GRD")
                    .filterDate(start_date, end_date)
                    .filter(ee.Filter.eq("instrumentMode", "IW"))
                    .filterMetadata("resolution_meters", "equals", 10)
                    .filterBounds(extent)
                    .select(["VV", "VH"]))
            size = img_collection.size().getInfo()
            print(f"There are {size} Sentinel-1 Images")
            if size == 0:
                raise Exception("No Sentinel-1 images found for the specified date range and region.")
            return img_collection

        def speckle(image):
            image_filtered = image.focal_median(radius, "circle", "meters")
            return image_filtered.copyProperties(image, image.propertyNames())
        
        def ratio(image):
            vh = image.select("VH")
            vv = image.select("VV")
            ratio = vh.divide(vv).rename("VH_VV_ratio")
            return image.addBands([ratio])

        s1_collection = imgdownload()
        s1_collection_filtered = (s1_collection
                                  .map(speckle)
                                  .map(ratio))
        s1_median = s1_collection_filtered.median().clip(extent)
        
        print(f"The bands in s1_median are: {s1_median.bandNames().getInfo()}")
        return s1_median
    except Exception as e:
        print(f"Error in Sentinel-1 processing: {e}")
        raise

try:
    s2_median_2023 = s2Process(dates=(start_date, end_date), extent=roi, cloud=50)
    s1_median_2023 = s1Process(dates=(start_date, end_date), extent=roi, radius=50)
    stacked_layer = s2_median_2023.select(['ndvi', 'mndwi']).addBands(s1_median_2023.select(['VH', 'VV', 'VH_VV_ratio']))
    print(f"The bands in the stacked layer are: {stacked_layer.bandNames().getInfo()}")
except Exception as e:
    print(f"Error processing Sentinel images: {e}")
    exit()

# 4. Generate Training and Testing Samples
def generate_non_water_samples(roi, water_polygons, num_points):
    """Generate random non-water points within ROI but outside water polygons."""
    try:
        points = ee.FeatureCollection([])
        while points.size().getInfo() < num_points:
            random_points = ee.FeatureCollection.randomPoints(
                region=roi.geometry(), 
                points=num_points, 
                seed=np.random.randint(1, 100000)
            )
            non_water_points = random_points.filter(ee.Filter.geometry(water_polygons.geometry().dissolve()).Not())
            non_water_points_classified = non_water_points.map(lambda f: f.set('Class', 0))
            points = points.merge(non_water_points_classified)
        return points.limit(num_points)
    except Exception as e:
        print(f"Error generating non-water samples: {e}")
        raise

# Generate water samples as random points inside water polygons
try:
    water_union = water_polygons_ee.geometry().dissolve()
    water_samples = ee.FeatureCollection.randomPoints(
        region=water_union,
        points=M,
        seed=np.random.randint(1, 100000)
    ).map(lambda f: f.set('Class', 1))
    print(f"Water samples created (labeled as 1): {water_samples.size().getInfo()}")
except Exception as e:
    print(f"Error generating water samples: {e}")
    exit()

# Generate equal number of non-water samples
try:
    non_water_samples = generate_non_water_samples(roi, water_polygons_ee, M)
    print(f"Non-water samples created (labeled as 0): {non_water_samples.size().getInfo()}")
except Exception as e:
    print(f"Error generating non-water samples: {e}")
    exit()

try:
    combined_samples = water_samples.merge(non_water_samples)
    print(f"Total training samples created: {combined_samples.size().getInfo()}")
except Exception as e:
    print(f"Error combining samples: {e}")
    exit()

def trainTest(sample, image, threshold, output_folder="Outputs", timeout=1200, max_retries=3):
    """Split samples into training and testing sets and sample image values."""
    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        sample_random = sample.randomColumn("random")
        training_set_features = sample_random.filter(ee.Filter.lt("random", threshold))
        testing_set_features = sample_random.filter(ee.Filter.gte("random", threshold))
        
        # Sample image values at points
        def sample_point(feature):
            class_value = feature.get('Class')
            sampled_values = image.reduceRegion(
                reducer=ee.Reducer.first(),
                geometry=feature.geometry(),
                scale=30  # Increased scale to reduce computation load
            )
            return ee.Feature(feature.geometry(), sampled_values).set('Class', class_value)

        training_samples = training_set_features.map(sample_point)
        testing_samples = testing_set_features.map(sample_point)
        
        # Export training and testing sets to local CSV
        training_file = os.path.join(output_folder, "training_samples.csv")
        testing_file = os.path.join(output_folder, "testing_samples.csv")
        
        for attempt in range(max_retries):
            try:
                print(f"Attempting to export training samples (Attempt {attempt + 1}/{max_retries})...")
                start_time = time.time()
                geemap.ee_export_table(
                    training_samples,
                    filename=training_file,
                    timeout=timeout
                )
                print(f"Training samples exported to {training_file}")
                
                print(f"Attempting to export testing samples (Attempt {attempt + 1}/{max_retries})...")
                geemap.ee_export_table(
                    testing_samples,
                    filename=testing_file,
                    timeout=timeout
                )
                print(f"Testing samples exported to {testing_file}")
                
                break
            except Exception as e:
                print(f"Export attempt {attempt + 1} failed: {e}")
                if attempt + 1 == max_retries:
                    raise Exception(f"Failed to export samples after {max_retries} attempts: {e}")
                time.sleep(10)  # Wait before retrying
        
        # Load exported CSV files
        training_df = pd.read_csv(training_file)
        testing_df = pd.read_csv(testing_file)
        
        # Convert back to EE FeatureCollections
        def df_to_ee_fc(df):
            features = []
            for _, row in df.iterrows():
                geom = ee.Geometry.Point([row['longitude'], row['latitude']])
                properties = {col: row[col] for col in df.columns if col not in ['longitude', 'latitude', 'system:index', '.geo']}
                feature = ee.Feature(geom, properties)
                features.append(feature)
            return ee.FeatureCollection(features)
        
        training = df_to_ee_fc(training_df)
        testing = df_to_ee_fc(testing_df)
        
        print(f"Training set size: {training.size().getInfo()}")
        print(f"Testing set size: {testing.size().getInfo()}")

        return training, testing
    except Exception as e:
        print(f"Error in trainTest: {e}")
        raise

def filter_invalid_samples(feature_collection, band_names, class_property='Class'):
    all_properties_to_check = band_names + [class_property]
    combined_filter = ee.Filter.And(
        [ee.Filter.notNull([prop]) for prop in all_properties_to_check]
    )
    filtered = feature_collection.filter(combined_filter)
    counts = filtered.aggregate_histogram(class_property).getInfo()
    print(f"Class distribution after filtering: {counts}")
    if len(counts) < 2:
        raise ValueError(f"Filtered dataset contains only one class: {counts}")
    return filtered

try:
    training, testing = trainTest(combined_samples, stacked_layer, 0.7, timeout=1200)  # Timeout set to 20 minutes
    band_names = stacked_layer.bandNames().getInfo()
    training = filter_invalid_samples(training, band_names)
    testing = filter_invalid_samples(testing, band_names)
    print("Training set class distribution:", training.aggregate_histogram('Class').getInfo())
    print("Testing set class distribution:", testing.aggregate_histogram('Class').getInfo())
except Exception as e:
    print(f"Error in trainTest: {e}")
    exit()

# 5. Classification and Accuracy Assessment
def rf_svm_classify(classifier_type, parameters, image, training, testing, out_folder, out_file_base):
    try:
        if not os.path.exists(out_folder):
            os.makedirs(out_folder)
        
        error_file_name = os.path.join(out_folder, out_file_base + "error_matrix" + ".csv")
        
        if classifier_type == "rf":
            trees, split, leaf, bag, node, seed = parameters
            model = ee.Classifier.smileRandomForest(
                numberOfTrees=trees,
                variablesPerSplit=split,
                minLeafPopulation=leaf,
                bagFraction=bag, 
                maxNodes=node,
                seed=seed
            )
            classifier_name = "Random Forest"
        elif classifier_type == "svm":
            kernelType, gamma, cost = parameters
            model = ee.Classifier.libsvm(
                kernelType=kernelType,
                gamma=gamma,
                cost=cost
            )
            classifier_name = "SVM"
        
        classifier = model.train(
            features=training, 
            classProperty="Class", 
            inputProperties=image.bandNames()
        )

        validation = testing.classify(classifier)
        
        prediction_counts = validation.aggregate_histogram('classification').getInfo()
        print(f"Prediction distribution for {classifier_name}: {prediction_counts}")
        if len(prediction_counts) < 2:
            raise ValueError(f"Classifier predicted only one class: {prediction_counts}")

        error_matrix = validation.errorMatrix(actual="Class", predicted="classification")
        error_matrix_array = error_matrix.array().getInfo()
        print(f"Raw error matrix array for {classifier_name}:", error_matrix_array)
        
        if len(error_matrix_array) < 2 or any(len(row) < 2 for row in error_matrix_array):
            raise ValueError(f"Error matrix has unexpected shape: {error_matrix_array}")
        
        columns = ["Non-Water", "Water"]
        try:
            data = error_matrix_array
            error_matrix_df = pd.DataFrame(data, columns=columns, index=columns)
        except Exception as e:
            print(f"Error creating DataFrame: {e}")
            print("Falling back to raw error matrix data.")
            error_matrix_df = pd.DataFrame(error_matrix_array)
            
        print(f"\nConfusion matrix for {classifier_name}:\n", error_matrix_df)
        print(f"Overall Accuracy for {classifier_name}:", round(error_matrix.accuracy().getInfo(), 3)) 
        print(f"Kappa for {classifier_type}:", round(error_matrix.kappa().getInfo(), 3))
        
        error_matrix_df.to_csv(error_file_name)
        print(f"Error matrix saved to {error_file_name}")
        
        return image.classify(classifier), error_matrix_df
    except Exception as e:
        print(f"Error in {classifier_type} classification: {e}")
        raise

rf_classification = None
rf_parameters = (50, None, 1, 0.5, None, 0)
try:
    rf_classification, rf_confusion_matrix = rf_svm_classify(
        "rf", 
        rf_parameters, 
        stacked_layer, 
        training, 
        testing, 
        "Outputs", 
        "rf_binary_classification_"
    )
except Exception as e:
    print(f"Error in Random Forest classification: {e}")
    exit()

if rf_classification is None:
    print("Error: rf_classification could not be created. Exiting.")
    exit()

svm_classification = None
svm_parameters = ('RBF', 0.1, 1)
try:
    svm_classification, svm_confusion_matrix = rf_svm_classify(
        "svm", 
        svm_parameters, 
        stacked_layer, 
        training, 
        testing, 
        "Outputs", 
        "svm_binary_classification_"
    )
except Exception as e:
    print(f"Error in SVM classification: {e}")
    exit()

if svm_classification is None:
    print("Error: svm_classification could not be created. Exiting.")
    exit()

# 6. Count Pixels Classified as Water
def count_water_pixels(classification_image, region, scale=10):
    """Count the number of pixels classified as water (Class 1) in the image."""
    try:
        water_mask = classification_image.eq(1)
        pixel_count = water_mask.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=region.geometry(),
            scale=scale,
            maxPixels=1e9
        ).get('classification')
        count = pixel_count.getInfo()
        if count is None:
            print("Warning: No pixels classified as water.")
            return 0
        return count
    except Exception as e:
        print(f"Error counting water pixels: {e}")
        return 0

try:
    rf_water_pixels = count_water_pixels(rf_classification, roi)
    svm_water_pixels = count_water_pixels(svm_classification, roi)
    print(f"Number of pixels classified as water by Random Forest: {rf_water_pixels}")
    print(f"Number of pixels classified as water by SVM: {svm_water_pixels}")
except Exception as e:
    print(f"Error in counting water pixels: {e}")

# 7. Export Original and Predicted Data as TIFF
output_folder = "Outputs"
try:
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    max_retries = 3
    timeout = 1200  # Default timeout for TIFF exports (20 minutes)
    
    # Export ground truth water raster
    ground_truth_file = os.path.join(output_folder, "ground_truth_water.tif")
    for attempt in range(max_retries):
        try:
            print(f"Attempting to export ground truth TIFF (Attempt {attempt + 1}/{max_retries})...")
            geemap.ee_export_image(
                water_image,
                filename=ground_truth_file,
                scale=10,
                region=roi.geometry(),
                file_per_band=False,
                timeout=timeout
            )
            print(f"Ground truth water raster exported to {ground_truth_file}")
            break
        except Exception as e:
            print(f"Ground truth export attempt {attempt + 1} failed: {e}")
            if attempt + 1 == max_retries:
                raise Exception(f"Failed to export ground truth TIFF after {max_retries} attempts: {e}")
            time.sleep(10)
    
    # Export Random Forest classification
    rf_file = os.path.join(output_folder, "rf_classification.tif")
    for attempt in range(max_retries):
        try:
            print(f"Attempting to export Random Forest TIFF (Attempt {attempt + 1}/{max_retries})...")
            geemap.ee_export_image(
                rf_classification,
                filename=rf_file,
                scale=10,
                region=roi.geometry(),
                file_per_band=False,
                timeout=timeout
            )
            print(f"Random Forest classification exported to {rf_file}")
            break
        except Exception as e:
            print(f"Random Forest export attempt {attempt + 1} failed: {e}")
            if attempt + 1 == max_retries:
                raise Exception(f"Failed to export Random Forest TIFF after {max_retries} attempts: {e}")
            time.sleep(10)
    
    # Export SVM classification
    svm_file = os.path.join(output_folder, "svm_classification.tif")
    for attempt in range(max_retries):
        try:
            print(f"Attempting to export SVM TIFF (Attempt {attempt + 1}/{max_retries})...")
            geemap.ee_export_image(
                svm_classification,
                filename=svm_file,
                scale=10,
                region=roi.geometry(),
                file_per_band=False,
                timeout=timeout
            )
            print(f"SVM classification exported to {svm_file}")
            break
        except Exception as e:
            print(f"SVM export attempt {attempt + 1} failed: {e}")
            if attempt + 1 == max_retries:
                raise Exception(f"Failed to export SVM TIFF after {max_retries} attempts: {e}")
            time.sleep(10)

except Exception as e:
    print(f"Error exporting TIFF files: {e}")
    exit()

# 8. Visualize the final classification and original data
palette_ground_truth = ['b4b4b4', '0000ff']  # Gray for Non-Water, Blue for Ground Truth Water
palette_rf = ['b4b4b4', '00ffff']  # Gray for Non-Water, Cyan for RF Water
palette_svm = ['b4b4b4', '00ff00']  # Gray for Non-Water, Green for SVM Water
vis_params_ground_truth = {'min': 0, 'max': 1, 'palette': palette_ground_truth}
vis_params_rf = {'min': 0, 'max': 1, 'palette': palette_rf}
vis_params_svm = {'min': 0, 'max': 1, 'palette': palette_svm}

try:
    Map = geemap.Map(center=map_center, zoom=10)
    
    Map.addLayer(water_polygons_ee, {'color': '0000ff', 'fillColor': '0000ff', 'opacity': 0.5}, "Ground Truth Water")
    Map.addLayer(rf_classification, vis_params_rf, "Water Classification (RF)")
    Map.addLayer(svm_classification, vis_params_svm, "Water Classification (SVM)")
    Map.addLayer(roi.geometry(), {'color': 'ff0000', 'fillColor': '00000000'}, "Study Area Boundary")
    Map.add_basemap("SATELLITE")
    Map.add_legend(
        title="Water Classification",
        legend_dict={
            'Non-Water': 'b4b4b4',
            'Ground Truth Water': '0000ff',
            'RF Predicted Water': '00ffff',
            'SVM Predicted Water': '00ff00'
        }
    )
    Map
except Exception as e:
    print(f"Error visualizing map: {e}")
    exit()

Earth Engine initialized successfully.
Converting 3D geometries to 2D...
Validating and cleaning geometries...
Invalid geometries in water_areas_gdf: 0
Empty geometries in water_areas_gdf: 0
Number of features in water_areas_gdf after cleaning: 1078
Geometry types in water_areas_gdf: ['Polygon']
Invalid geometries in study_area_gdf: 0
Empty geometries in study_area_gdf: 0
Number of features in study_area_gdf after cleaning: 1
The center point of your study area is: Latitude=38.2285, Longitude=45.9499
Successfully converted study_area_gdf to Earth Engine FeatureCollection.
Successfully converted water_areas_gdf to Earth Engine FeatureCollection.
Calculating the number of 10m pixels observed as water...
Number of 10m pixels observed as water: 2332
There are 103 Sentinel-2 Images
The bands in s2_median are: ['B2', 'B3', 'B4', 'B8', 'B11', 'ndvi', 'mndwi']
There are 59 Sentinel-1 Images
The bands in s1_median are: ['VV', 'VH', 'VH_VV_ratio']
The bands in the stacked layer are: ['ndvi', 'mn

: 